<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/02_dnn_simple_pytorch/MNIST_CNN_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNによる画像認識（MNIST, PyTorch実装）


---
## 目的
PyTorch実装による畳み込みニューラルネットワーク（CNN）を用いてMNISTデータセットに対する文字認識を行う．
評価はConfusion Matrixにより各クラスの認識率を用いて行う．

また，GPUを用いたネットワークの計算を行う．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**

## モジュールのインポート
はじめに必要なモジュールをインポートする．

In [ ]:
from time import time   # 時間計測を行うモジュール

import numpy as np      # Pythonで配列を扱うためのモジュール
import torch            # 深層学習のモジュール（PyTorch）
import torch.nn as nn

import torchvision      # PyTorchで画像認識に使用するネットワークやデータセットを利用するためのモジュール
import torchvision.transforms as transforms

import torchsummary     # PyTorchで作成したネットワークモデルの詳細を表示するモジュール

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込みと確認
学習データ（MNIST Dataset）を読み込みます．

In [ ]:
train_data = torchvision.datasets.MNIST(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.MNIST(root="./", train=False, transform=transforms.ToTensor(), download=True)

### 読み込んだデータの情報を表示して確認してみます
# 画像・ラベルデータのデータタイプ（型）
print(type(train_data.data), type(train_data.targets))
print(type(test_data.data), type(test_data.targets))
# 画像・ラベルの配列サイズ
print(train_data.data.size(), train_data.targets.size())
print(test_data.data.size(), test_data.targets.size())

## ネットワークモデルの定義

畳み込みニューラルネットワークを定義します．

ここでは，畳み込み層1層，全結合層1層から構成されるネットワークとします．

畳み込み層は入力チャンネル数が1，出力する特徴マップ数が4，畳み込むフィルタサイズが3x3です．
全結合層は入力ユニット数は`14*14*4`とし，出力は10としています．
また，活性化関数として`self.act`にシグモイド関数を定義します．
さらに，プーリング処理を行うための`self.pool`を定義します．
ここでは，maxpoolingを使用します．
これらの各層の構成を`__init__`関数で定義します．

次に，`forward`関数では，定義した層を接続して処理するように記述します．
`forward`関数の引数`x`は入力データです．
それを`__init__`関数で定義した`conv1`に入力し，その出力を活性化関数である`self.act`に与えます．
そして，その出力を`self.pool`に与えて，プーリング処理結果を`h`として出力します．

畳み込みを適用した後の特徴マップを全結合層へと入力して，識別結果を出力します．
まず．畳み込みによって得られた特徴マップの形状（チャンネルx縦x横）を1次元の配列へと変換します．
ここで，`view()`を用いることで，`h`の配列を操作します．引数として，変換したい配列のサイズを入力します．
まず一つ目の引数の`h.size()[0]`で，`h`の1次元目のサイズを取得し，変換後の配列の1次元目のサイズとして指定します．
二つ目の引数の`-1`で任意のサイズを指定します．
これにより，`h`を（バッチ数x任意の長さのデータ）の形状へ変換します．
変換した`h`を全結合層へと入力することで，最終的にクラススコアを返します．

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1)
        self.l1 = nn.Linear(14*14*4, 10)
        self.act = nn.Sigmoid()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):                       # 入力画像サイズ: [batch, 1, 28, 28]
        h = self.pool(self.act(self.conv1(x)))  # 出力される特徴マップサイズ: [batch, 4, 14, 14]
        # 特徴マップの配列サイズを [batch, channel, height, width] --> [batch, channel * height * width] に変更
        h = h.view(h.size()[0], -1)
        h = self.l1(h)
        return h

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．

`CNN`クラスを呼び出して，ネットワークモデルを定義します．
また，GPUを使う場合（`use_cuda == True`）には，ネットワークモデルをGPUメモリ上に配置します．
これにより，GPUを用いた演算が可能となります．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．
また，学習率を0.01，モーメンタムを0.9として引数に与えます．

最後に，定義したネットワークの詳細情報を`torchsummary.summary()`関数を用いて表示します．

In [ ]:
### ネットワークの作成
model = CNN()
if use_cuda:      # GPUが使用できる場合は
    model.cuda()  # ネットワークの演算をGPUで行うように設定変更 (デフォルトはCPU)

### 最適化手法の設定 lr: 学習率, momentum: モーメンタム (慣性項のパラメータ)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### 定義したモデルの情報を表示
if use_cuda:
    torchsummary.summary(model, (1, 28, 28), device='cuda')  # GPUで演算する設定の場合
else:
    torchsummary.summary(model, (1, 28, 28), device='cpu')   # CPUで演算する設定の場合

## 学習
読み込んだMNISTデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を100，学習エポック数を10とします．

次にデータローダーを定義します．
データローダーでは，上で読み込んだデータセット（`train_data`）を用いて，for文で指定したミニバッチサイズでデータを読み込むオブジェクトを作成します．
この時，`shuffle=True`と設定することで，読み込むデータを毎回ランダムに指定します．

次に，誤差関数を設定します．
今回は，分類問題をあつかうため，クロスエントロピー誤差を計算するための`CrossEntropyLoss`を`criterion`として定義します．

学習を開始します．

各更新において，学習用データと教師データをそれぞれ`image`と`label`とします．
学習モデルにimageを与えて各クラスの確率yを取得します．
各クラスの確率yと教師ラベルtとの誤差を`criterion`で算出します．
また，認識精度も算出します．
そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 100
epoch_num = 10

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定 (必要に応じて誤差関数の計算もGPUで行うように設定変更)
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

# 学習の実行
train_start = time()
for epoch in range(1, epoch_num+1):   # epochのforループ
    # 1 epochの学習中の誤差・学習画像が正解した数をカウントする変数を初期化
    sum_loss = 0.0
    count = 0

    for image, label in train_loader:  # 1 epoch内のforループ (iterationのループ)

        if use_cuda:  # GPUで計算する場合は，データもGPUメモリ上へ移動させる
            image = image.cuda()
            label = label.cuda()

        y = model(image)  # データの入力と結果の出力

        # 誤差計算とbackpropagation, パラメータの更新
        loss = criterion(y, label)
        model.zero_grad()
        loss.backward()
        optimizer.step()

        # 学習経過を確認するための処理
        sum_loss += loss.item()
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    # 1 epoch終了時点での誤差の平均値，学習データに対する認識精度, 学習開始からの経過時間を表示
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed time: {}".format(epoch, sum_loss/len(train_loader), count.item()/len(train_data), time() - train_start))

## テスト

学習したネットワークを用いて，テストデータに対する認識率の確認を行います．

`model.eval()`を適用することで，ネットワーク演算を評価モードへ変更します．
これにより，学習時と評価時で挙動が異なる演算（dropout等）を変更することが可能です．
また，`torch.no_grad()`を適用することで，学習時には必要になる勾配情報を保持することなく演算を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():  # 勾配を計算しない設定にする (loss.backwardをしても勾配情報が計算されない)
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## 課題


### 1. ネットワークの構造を変更し，認識精度の変化を確認しましょう．

**ヒント：ネットワーク構造の変更としては，次のようなものが考えられます．**
* 中間層のユニット数
* 層の数
* 活性化関数
  * `nn.Tanh()`や`nn.ReLU()`, `nn.LeakyReLU()`などが考えられます．
  * その他のPyTorchで使用できる活性化関数は[こちらページ](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)にまとめられています．

※ ネットワーク構造を変更した際には，`torchsummary.summary(***)`を使用し，ネットワーク構造を変更した際のパラメータ数の変化を確認してみましょう．